In [1]:
import pynq
from pynq import Overlay
from pynq.lib.dma import DMA
import numpy as np
from pynq import allocate

overlay = Overlay("add.bit")
dma = overlay.axi_dma_0

t = [1, 2, 3, 4, 5, 6, 7]
n = len(t)

array_t = np.array(t)

sample = array_t.astype(np.int32)

# Allocate buffers for the input and output signals
with allocate(shape=(n,), dtype=np.int32) as in_buffer, \
     allocate(shape=(n,), dtype=np.int32) as out_buffer:
    np.copyto(in_buffer,sample)
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    print(in_buffer)
    print(out_buffer)

[1 2 3 4 5 6 7]
[ 1  3  6 10 15 21 28]


In [2]:
# Import necessary libraries from PYNQ framework
import pynq
from pynq import Overlay
from pynq.lib.dma import DMA
import numpy as np
from pynq import allocate
import time

# Load the hardware overlay (bitstream) that contains the design with DMA and custom adder IP
overlay = Overlay("design1_second_working.bit")
dma = overlay.axi_dma_0

# Generate an array of 10,000 random integers between 1 and 5000 to be summed
random_numbers = np.random.randint(1, 100, size=1000000)

# Assign the generated random numbers to variable 't'
t = random_numbers

# Number of elements in the array
n = len(t)
print(n)
# Initialize variables to store sum, software execution time, and hardware execution time
sum_cpu = 0
sw_exe_time = 0
hw_exe_time = 0

# -------- CPU (Software) Execution ---------
# Measure the execution time of summing the array using the CPU
start_time = time.time()
for i in range(n):
    sum_cpu = sum_cpu + t[i]  # Accumulate the sum of all numbers
stop_time = time.time()

# Calculate the CPU execution time for summing 10,000 numbers
sw_exe_time = stop_time - start_time

# Convert the array to a numpy array with int32 data type
array_t = np.array(t)
sample = array_t.astype(np.int32)

# -------- FPGA (Hardware) Execution ---------
# Allocate memory buffers for the input data (to FPGA) and output data (from FPGA)
with allocate(shape=(n,), dtype=np.int32) as in_buffer, \
     allocate(shape=(1,), dtype=np.int32) as out_buffer:
    
    # Copy the input data into the allocated input buffer
    np.copyto(in_buffer, sample)

    # Start measuring the FPGA execution time
    start_time = time.time()

    # Transfer the input buffer to the FPGA via DMA
    dma.sendchannel.transfer(in_buffer)
    # Transfer the output buffer to receive the result from FPGA
    dma.recvchannel.transfer(out_buffer)

    # Wait for both DMA send and receive channels to complete the transfers
    dma.sendchannel.wait()
    dma.recvchannel.wait()

    # Measure the FPGA execution time
    stop_time = time.time()
    hw_exe_time = stop_time - start_time

    # Print the output result (sum) from the FPGA
    print("FPGA Result (Sum):", out_buffer[0])

    # Close the buffers after use
    in_buffer.close()
    out_buffer.close()

# Print the execution times and performance comparison
print("FPGA (sum of 10,000 numbers) Execution Time: ", hw_exe_time, "seconds")

print("CPU Result (Sum):", sum_cpu)
print("CPU (sum of 10,000 numbers) Execution Time: ", sw_exe_time, "seconds")

print("Performance Gain: FPGA is", sw_exe_time / hw_exe_time, "times faster than CPU")

1000000


RuntimeError: DMA channel not started

In [25]:
import pynq
from pynq import Overlay
from pynq.lib.dma import DMA
import numpy as np
from pynq import allocate
import time

# overlay = Overlay("design1.bit")
overlay = Overlay("sumAll.bit")

dma = overlay.axi_dma_0

import numpy as np

# random_numbers = list(range(1, 1000000 + 1))
random_numbers = np.random.randint(1, 10, size=1000000)


# print(random_numbers)

# t = [1, 2, 3, 4, 5, 6, 7]
t = random_numbers

n = len(t)
print(n)
sum = 0
sw_exe_time = 0
hw_exe_time = 0
# start_time = time.time()
# for i in range(n):
#     sum = sum + t[i];

# stop_time = time.time()
# sw_exe_time =  sw_exe_time + stop_time - start_time
# print("cpu result", sum)
array_t = np.array(t)



sample = array_t.astype(np.int32)



# Allocate buffers for the input and output signals
with allocate(shape=(n,), dtype=np.int32) as in_buffer, \
     allocate(shape=(1,), dtype=np.int32) as out_buffer:
    np.copyto(in_buffer,sample)
    start_time = time.time()
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    stop_time = time.time()
    hw_exe_time = hw_exe_time + stop_time - start_time
#     print(in_buffer)
    print(out_buffer)
    in_buffer.close()
    out_buffer.close()
    
print("CPU(sum 1000 numbers) ", sw_exe_time)
print("FPGA(sum 1000 numbers) ", hw_exe_time)
print("FPGA faster ", sw_exe_time/hw_exe_time)

print(sum)

1000000
[5001352]
CPU(sum 1000 numbers)  0
FPGA(sum 1000 numbers)  0.018686532974243164
FPGA faster  0.0
0


In [16]:
import pynq
from pynq import Overlay
from pynq.lib.dma import DMA
import numpy as np
from pynq import allocate
import time

# Load the overlay
overlay = Overlay("sumAll.bit")
dma = overlay.axi_dma_0

In [10]:
# Generate random numbers array
random_numbers = np.random.randint(1, 10, size=100000000)  # Large data array

# Define manageable chunk size (adjust based on memory constraints)
chunk_size = 1000000  # Try smaller sizes if memory issues persist


In [17]:
n = len(random_numbers)
print(n)

100000000


In [18]:
num_chunks = n // chunk_size + (1 if n % chunk_size != 0 else 0)  # Total number of chunks
print(num_chunks)

100


In [15]:
# Initialize variables to store results and time
total_fpga_sum = 0
hw_exe_time = 0

# Process each chunk
for i in range(num_chunks):
    print(i)
    start = i * chunk_size
    end = min((i + 1) * chunk_size, n)
    chunk = random_numbers[start:end].astype(np.int32)  # Convert chunk to int32 for DMA

    # Allocate buffers for input and output for each chunk
    with allocate(shape=(len(chunk),), dtype=np.int32) as in_buffer, \
         allocate(shape=(1,), dtype=np.int32) as out_buffer:
        
        # Copy data to input buffer
        np.copyto(in_buffer, chunk)
        
        # FPGA Execution
        start_time = time.time()
        dma.sendchannel.transfer(in_buffer)
        dma.recvchannel.transfer(out_buffer)
        dma.sendchannel.wait()
        dma.recvchannel.wait()
        stop_time = time.time()
        
        # Add execution time
        hw_exe_time += (stop_time - start_time)
        
        # Accumulate the FPGA result
        total_fpga_sum += out_buffer[0]  # Add FPGA's partial sum to the total sum

0


RuntimeError: DMA channel not idle

In [20]:
#  Define manageable chunk size (adjust based on memory constraints)
chunk_size = 1000000  # Try smaller sizes if memory issues persist
n = len(random_numbers)
num_chunks = (n + chunk_size - 1) // chunk_size  # Calculate total number of chunks
print(n, num_chunks, chunk_size)
# Initialize variables to store results and time
total_fpga_sum = 0
hw_exe_time = 0

100000000 100 1000000


In [21]:
for i in range(num_chunks):
    print(i)
    start = i * chunk_size
    end = min((i + 1) * chunk_size, n)
    chunk = random_numbers[start:end].astype(np.int32)  # Convert chunk to int32 for DMA

    # Allocate buffers for input and output for each chunk
    with allocate(shape=(len(chunk),), dtype=np.int32) as in_buffer, \
         allocate(shape=(1,), dtype=np.int32) as out_buffer:
        
        # Copy data to input buffer
        np.copyto(in_buffer, chunk)
        
        # FPGA Execution
        start_time = time.time()
        dma.sendchannel.transfer(in_buffer)
        dma.recvchannel.transfer(out_buffer)
        dma.sendchannel.wait()
        dma.recvchannel.wait()
        stop_time = time.time()
        
        # Accumulate execution time and result
        hw_exe_time += (stop_time - start_time)
        total_fpga_sum += int(out_buffer[0])  # Convert to int to avoid unexpected types

0
1


KeyboardInterrupt: 

In [22]:
import pynq
from pynq import Overlay
from pynq.lib.dma import DMA
import numpy as np
from pynq import allocate
import time

# Load the overlay
overlay = Overlay("sumAll.bit")
dma = overlay.axi_dma_0

# Generate random numbers array
random_numbers = np.random.randint(1, 10, size=100000000)  # Large data array

# Define manageable chunk size (reduce if necessary)
chunk_size = 500000  # Smaller chunk size to manage memory constraints
n = len(random_numbers)
num_chunks = (n + chunk_size - 1) // chunk_size  # Calculate total number of chunks

# Initialize variables to store results and time
total_fpga_sum = 0
hw_exe_time = 0

# Process each chunk
for i in range(num_chunks):
    start = i * chunk_size
    end = min((i + 1) * chunk_size, n)
    chunk = random_numbers[start:end].astype(np.int32)  # Convert chunk to int32 for DMA
    
    print(f"Processing chunk {i+1}/{num_chunks}, size: {len(chunk)}")  # Debug output

    # Allocate buffers for input and output for each chunk
    with allocate(shape=(len(chunk),), dtype=np.int32) as in_buffer, \
         allocate(shape=(1,), dtype=np.int32) as out_buffer:
        
        # Copy data to input buffer
        np.copyto(in_buffer, chunk)
        
        # FPGA Execution
        start_time = time.time()
        dma.sendchannel.transfer(in_buffer)
        dma.recvchannel.transfer(out_buffer)
        
        # Ensure DMA synchronization
        dma.sendchannel.wait()
        dma.recvchannel.wait()
        
        stop_time = time.time()
        
        # Accumulate execution time and result
        hw_exe_time += (stop_time - start_time)
        total_fpga_sum += int(out_buffer[0])  # Convert to int to avoid unexpected types

        print(f"Chunk {i+1} processed. Partial FPGA sum: {total_fpga_sum}")  # Debug output

# CPU Execution (Software) for comparison
start_time = time.time()
cpu_sum = np.sum(random_numbers)  # CPU sum for the full array
stop_time = time.time()
sw_exe_time = stop_time - start_time

# Display results
print("CPU result:", cpu_sum)
print("FPGA result:", total_fpga_sum)
print("CPU Execution Time (seconds):", sw_exe_time)
print("FPGA Execution Time (seconds):", hw_exe_time)
if hw_exe_time > 0:
    print("FPGA speedup:", sw_exe_time / hw_exe_time)


Processing chunk 1/200, size: 500000
Chunk 1 processed. Partial FPGA sum: 2498556
Processing chunk 2/200, size: 500000


KeyboardInterrupt: 